# data movement

In [1]:
from random import randint, sample, choice
from avrasm import *

In [2]:
doc = """; Tests the data movement instructions

; Note, we attempt to verify the behavior of the load instructions
; by writing registers out with STS-
; the load tests assume STS is functional.
"""

## load instructions

In [3]:
def checkreg(reg, val):
    """check that a register has expected value by writing it out"""
    return STS(a(0), r(reg)).write(val, 0)(f"r{reg} should be {val}")

In [4]:
def testLoadI(values):
    """Tests LDI instruction with a list of
    (register, value)
    """
    loads = [LDI(r(reg), v(value)) for (reg, value) in values]
    stores = [checkreg(reg, value) for (reg, value) in values]
    return loads + stores

In [5]:
ldiTests = [
    "\n\n;PREPROCESS TestLDI",
    "; do a few simple LDIs",
    testLoadI([(16, 0x00)]),
    testLoadI([(17, 0xFF)]),
    testLoadI([(18, 0x0F)]),
    "; do a fixed LDI into all 16 upper registers",
    testLoadI([(i, 0xEC) for i in range(16,32)]),
    "; do some random LDIs",
    testLoadI([(reg, randint(0,0xFF)) for reg in sample(range(16,32),10)])
]

In [6]:
def testLoad(values):
    """Tests LD instruction with list of
    (target register, address register, value, address)
    """
    loads = [LD(r(treg), areg).read(value, addr)
                 for (treg, areg, value, addr) in values]
    stores = [checkreg(treg, value) for (treg, _, value, _) in values]
    return loads + stores

In [7]:
regs = {"X": (26,27), "Y": (28,29), "Z": (30,31)}
sampleRegs = lambda n, ex: sample(list(set(range(32)) - set(regs[ex])), n)
def setWord(reg, value):
    low, high = regs[reg]
    return [
        LDI(r(low), v(value%0x100))(f"set {reg} to {a(value)}"),
        LDI(r(high), v(value//0x100))
    ]

def testLoadVariants(reg):
    """Test all variants of LD with the X, Y, or Z register"""
    return [
        f"; do a few simple LDs from {reg} register",
        setWord(reg, 0x0004),
        testLoad([(0, reg, 0xEC, 0x0004)]),
        testLoad([(8, reg, 0xEC, 0x0004)]),
        testLoad([(31, reg, 0xEC, 0x0004)]),
        "; load with pre decrement through 0x0000",
        testLoad([(r, f"-{reg}", randint(0,0xFF), (0x0004-i-1)%0x10000) 
                  for i,r in enumerate(sampleRegs(8,reg))]),
        "; load with post increment through 0xFFFF",
        testLoad([(r, f"{reg}+", randint(0,0xFF), (0xFFFC+i)%0x10000) 
                  for i,r in enumerate(sampleRegs(8,reg))])
    ]

In [8]:
ldTests = [
    "\n\n;PREPROCESS TestLD\n",
    testLoadVariants("X"),
    testLoadVariants("Y"),
    testLoadVariants("Z"),
]

In [9]:
def testLoadD(values):
    """Test LDD instruction with Y or Z register
    (target register, address register, offset, value, actual address)
    """
    loads = [LDD(r(treg), f"{areg}+{off}").read(value, addr) 
                 for (treg, areg, off, value, addr) in values]
    stores = [checkreg(treg, value) for (treg, _, _, value, _) in values]
    return loads + stores

In [10]:
randReg = lambda ex: choice(list(set(range(32)) - set(regs[ex])))

def testLoadDReg(reg):
    """Test LDD instruction with Y or Z register"""
    offsets = [0] + [randint(0,63) for _ in range(6)] + [63]
    return [
        setWord(reg, 0xFFE0),
        testLoadD([(randReg(reg), reg, off, randint(0,0xFF), (0xFFE0+off)%0x10000)
              for off in offsets])
    ]

In [11]:
lddTests = [
    "\n\n;PREPROCESS TestLDD\n",
    testLoadDReg("Y"),
    testLoadDReg("Z"),
]

In [12]:
def testLoadS(values):
    """Tests LDS instruction with a list of
    (register, value, address)
    """
    loads = [LDS(r(reg), a(addr)).read(value, addr) for (reg, value, addr) in values]
    stores = [checkreg(reg, value) for (reg, value, checkaddr) in values]
    return loads + stores

In [13]:
ldsTests = [
    "\n\n;PREPROCESS TestLDS\n",
    "; do a few simple LDSs",
    testLoadS([(0, 0xEC, 0x0000)]),
    testLoadS([(15, 0xEC, 0xFF00)]),
    testLoadS([(30, 0xEC, 0xFFFF)]),
    "; do some random LDSs",
    testLoadS([(reg, randint(0,0xFF),randint(0,0xFFFF)) for reg in sample(range(32),10)]),
]

In [14]:
loadTests = [
    ldiTests, ldTests, lddTests, ldsTests
]

In [15]:
def testMove(regs):
    value = randint(0,0xFF)
    movs = [MOV(r(regs[i]), r(regs[i+1])) for i in range(len(regs)-1)]
    return [
        LDS(r(regs[0]), a(0x0000)).read(value, 0x0000),
        movs,
        checkreg(regs[-1], value)
    ]

In [16]:
movTests = [
    "\n\n;PREPROCESS TestMOV\n",
    "; do a few simple MOVs",
    testMove([0,1]),
    testMove([31,30]),
    "; do a random move through all registers",
    testMove(sample(range(32),32))
]

## store tests

In [17]:
def setreg(reg, val):
    return LDS(r(reg), a(0)).read(val, 0)(f"set r{reg} to {val}")

In [18]:
def testStore(values):
    """
    (source register, address register, value, address)
    """
    loads = [setreg(sreg, value) for (sreg, _, value, _) in values]
    stores = [ST(areg, r(sreg)).write(value, addr)
                     for (sreg, areg, value, addr) in values]
    return loads + stores

In [19]:
def testStoreVariants(reg):
    return [
        setWord(reg, 0x0004),
        testStore([(0, reg, 0xEC, 0x0004)]),
        testStore([(8, reg, 0xEC, 0x0004)]),
        testStore([(31, reg, 0xEC, 0x0004)]),
        "; store with pre decrement through 0x0000",
        testStore([(r, f"-{reg}", randint(0,0xFF), (0x0004-i-1)%0x10000) 
                  for i,r in enumerate(sampleRegs(8,reg))]),
        "; store with post increment through 0xFFFF",
        testStore([(r, f"{reg}+", randint(0,0xFF), (0xFFFC+i)%0x10000) 
                  for i,r in enumerate(sampleRegs(8,reg))])
    ]

In [20]:
stTests = [
    "\n\n;PREPROCESS TestST\n",
    testStoreVariants("X"),
    testStoreVariants("Y"),
    testStoreVariants("Z")
]

In [21]:
def testStoreD(values):
    """Test STD instruction with Y or Z register
    (source register, address register, offset, value, actual address)
    """
    loads = [setreg(r(sreg), value) for (sreg, _, _, value, _) in values]
    stores = [STD(f"{areg}+{off}", r(sreg)).write(value, addr)
                 for (sreg, areg, off, value, addr) in values]
    return loads + stores

In [22]:
def testStoreDReg(reg):
    """Test STD instruction with Y or Z register"""
    offsets = [0] + [randint(0,63) for _ in range(6)] + [63]
    return [
        setWord(reg, 0xFFE0),
        testStoreD([(randReg(reg), reg, off, randint(0,0xFF), (0xFFE0+off)%0x10000)
              for off in offsets])
    ]

In [23]:
stdTests = [ # ( ͡° ͜ʖ ͡°)
    "\n\n;PREPROCESS TestSTD\n",
    testStoreDReg("Y"),
    testStoreDReg("Z")
]

In [24]:
def testStoreS(values):
    """Tests STS instruction with a list of
    (register, value, address)
    """
    loads = [setreg(reg, value) for (reg, value, _) in values]
    stores = [STS(a(addr), r(reg)).write(value, addr)
                  for (reg, value, addr) in values]
    return loads + stores

In [25]:
stsTests = [
    "\n\n;PREPROCESS TestSTS\n",
    "; do a few simple STSs",
    testStoreS([(0, 0xEC, 0x0000)]),
    testStoreS([(15, 0xEC, 0xFF00)]),
    testStoreS([(30, 0xEC, 0xFFFF)]),
    "; do some random STSs",
    testStoreS([(reg, randint(0,0xFF),randint(0,0xFFFF)) for reg in sample(range(32),10)]),
]

In [26]:
storeTests = [stTests, stdTests, stsTests]

In [27]:
def testPushPop(regs):
    values = [randint(0,0xFF) for _ in regs]
    loads = [setreg(regs[i], values[i]) for i in range(len(regs))]
    pushes = [PUSH(r(regs[i])).write(values[i], (-i)%0x10000) for i in range(len(regs))]
    pops = [POP(r(regs[-i-1])).read(values[-i-1], (-len(regs)+i+1)%0x10000) for i in range(len(regs))]
    stores = [checkreg(regs[i], values[i]) for i in range(len(regs))]
    return [
        "; load some random values into registers", loads,
        "; push all the registers", pushes,
        "; pop all the registers", pops,
        "; check all the values are unchanged", stores
    ]

In [28]:
pushpopTests = [
    ";PREPROCESS TestPOP",
    ";PREPROCESS TestPOP",
    testPushPop(sample(range(32),32))
]

## output

In [29]:
datamoveTests = [
    doc,
    loadTests,
    movTests,
    storeTests,
    pushpopTests
]

In [30]:
with open("data_move_test.asm", "w") as f:
    f.write(stringify(datamoveTests))